# Dynamic Fill — 2 year observation window
### Prediction Imputation

- For this model we do not mask the training set's missing instances. We impute them.
- We do, however, perform masking for the teting data evaluation.

# Updates
- Remove all test operations
- mask_list.append((train, test)) went to mask_list.append(train)

# Libraries

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold

from keras import callbacks
import keras.layers as L
import keras.models as M
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error as mse
import os, shutil

from keras.regularizers import l1_l2

/Users/matiasberretta/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Functions

In [2]:
pot = 4
n_features = 115

def select_columns(col_list, n_months):
    """
    Takes in a list of column names and number of visits starting at 0.
    Returns column list time-stepped and dovetailed.
    """ 
    return dovetail_names(*[time_step_names(i, n_months) for i in col_list])
        
def time_step_names(name, n_months):

    return [(name + '_%d' % (j+1)) for j in range(-1,n_months*6, 6)]

def stretch_input(Xtr, time_steps, pot, n_features=n_features):
    """
    Xtr_fill is empty 3D numpy array where we extend length of patient observation times t
    pot stands for Patient Observation Time. We only need to do this for our X input
    """
    Xtr_fill = np.zeros(shape=[Xtr.shape[0],time_steps,n_features*pot] , dtype = object) 

    for subject in range(Xtr.shape[0]):
    
        for i in range(time_steps):
            
            concat_list = []
            
            for extra in range(pot):
                
                concat_list.append(Xtr[subject][i+extra])
                
            temp = np.concatenate(concat_list) 
            Xtr_fill[subject][i] = temp
            
    return Xtr_fill

def reshape_data(X, y, n_time_steps, pot=pot, n_features = n_features):  
    
    extra_ts = pot - 1
    
    X_reshaped = X.values.reshape(-1, n_time_steps+extra_ts, n_features)
    y_reshaped = y.values.reshape(-1, n_time_steps, 1)
    
    if (pot > 1):
        
        X = stretch_input(X_reshaped, n_time_steps, pot)

    y = y_reshaped.astype(float)
    X = X.astype(float)
    
    print("X reshaped is " + str(X.shape))
    print("y reshaped is " + str(y_reshaped.shape))
    
    return X, y

def provide_data(X, y, roll, n_features = n_features, pot =pot):
    
    extra_ts = pot - 1
    
    X = X.iloc[:,:(n_features*(roll+extra_ts))]
    y = y.iloc[:,:roll]

    y_full = y.dropna()

    mask = X.index.isin(y_full.index.tolist())

    X_full = X[mask]

    y_nan = y[~mask]
    X_nan = X[~mask]
    
    print('NaN')
    X_nan, y_nan = reshape_data(X_nan, y_nan, roll)
    print('Full')
    X_full, y_full = reshape_data(X_full, y_full, roll)
    
    return X_full, X_nan, y_full, y_nan, mask

def provide_all_data(X,y,roll, pot = pot):
 
    extra_ts = pot - 1
    
    X = X.iloc[:,:n_features*(roll+extra_ts)]
    y = y.iloc[:,:roll]

    X_all, y_all = reshape_data(X, y, roll)
    
    return X_all, y_all
    
def prepare_for_mask(X, y, mask_value = -99):
    """Improved and working"""
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if np.isnan(y[i][j][0]) or (y[i][j] == mask_value):

                X[i][j] = mask_value
                y[i][j] = mask_value
                
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if (mask_value in X[i][j]) or np.isnan(X[i][j]).any():
                
                X[i][j] = mask_value
                y[i][j] = mask_value
            
    return X,y

def round_off_EDSS(number):
    """Round a number to the closest half integer.
    >>> round_of_rating(1.3)
    1.5
    >>> round_of_rating(2.6)
    2.5
    >>> round_of_rating(3.0)
    3.0
    >>> round_of_rating(4.1)
    4.0"""
    return np.round(number * 2) / 2
   
def def_train_name(f_ix):
    
    X_train_name = "data_folds/X_train_f" + str(f_ix + 1) + ".csv"
    y_train_name = "data_folds/y_train_f" + str(f_ix + 1) + ".csv"
    X_test_name = "data_folds/X_test_f" + str(f_ix + 1) + ".csv"
    y_test_name = "data_folds/y_test_f" + str(f_ix + 1) + ".csv"
    
    return X_train_name, X_test_name, y_train_name, y_test_name

# Keras Callbacks

In [3]:
weight_file_path = "weights/my_model_weights_2y.h5"
final_file_path = "final_weights/my_model_weights_2y.h5"
best_file_path = "best/best_weights_2y.hdf5"

# reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                               patience=5, min_lr=0.001)

# stop training if there isn't a significant improvement in the course of 5 epochs
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, 
                              patience=5, verbose=0, mode='auto', 
                              baseline=None, restore_best_weights=True)

# model check point
model_checkpoint = ModelCheckpoint(best_file_path, monitor='val_loss', 
                                   verbose=1, 
                                   save_best_only=True, 
                                   mode='min')

callbacks_list = [reduce_lr, early_stopping]
callbacks_list_final = [reduce_lr, early_stopping, model_checkpoint]

In [4]:
def rnn_model(n_time_steps, n_inputs):
    
    m = M.Sequential()
    m.add(L.Masking(mask_value=-99, input_shape=(n_time_steps, n_inputs)))
    m.add(L.GRU(128, return_sequences=True))
    m.add(L.Dropout(0.2))
    m.add(L.Dense(1, activation='relu'))
    m.compile(optimizer = 'adam', loss = 'mean_absolute_error')

    return m

# Part 1 

- Unroll RNN one time step at a time
- Transfer weights between rolls

In [5]:
def generate_initial_predictions(n_time_steps, X_train, y_train, X_test, y_test, n_features=n_features, pot = pot):
    
    n_inputs = n_features*pot
    
    threshold = len(y_train.columns)
    print("TRAIN")
    X_full_train, X_nan_train, y_full_train, y_nan_train, mask_train = provide_data(X_train, y_train, n_time_steps)
    print("TEST")
    X_full_test, X_nan_test, y_full_test, y_nan_test, mask_test = provide_data(X_test, y_test, n_time_steps)
    
    K.clear_session()
    
    m = rnn_model(n_time_steps, n_inputs)

    if n_time_steps > 1:
        
        # load weights from previous model to establish continuity 
        m.load_weights(weight_file_path)
    
    history = m.fit(X_full_train, y_full_train, 
                    validation_split=0.2,
                    batch_size = X_full_train.shape[0], # batch size == size of the training data 
                    epochs=1, 
                    shuffle=True,
                    callbacks = callbacks_list)
        
    m.save_weights(weight_file_path) # save weights 
    
    # Predict for unknown values of training and testing data
    # Using train data with known targets, I train a model to predict for missing values
    # In both training and testing data 
    # But for the testing data, I only impute the X feature space 
    
    m_pred_train = round_off_EDSS(
        pd.Series(np.array([x[0] for x in m.predict(X_nan_train)]).flatten(),
                             index = y_train[~mask_train].index))
    
    # redundant but also helps me known which values to update for test data feature space
    m_pred_test = round_off_EDSS(pd.Series(np.array([x[0] for x in m.predict(X_nan_test)]).flatten(),
                           index = y_test[~mask_test].index))
    
    # In pandas the best way to update a dataframe or series
    # is with another dataframe or series with the desired index
    y_train.iloc[:, n_time_steps-1].update(m_pred_train)
    
    if (n_time_steps < threshold): # when y contains an EDSS column that X does not 
        
        X_train.iloc[:,(n_features*(n_time_steps+pot))-1].update(m_pred_train)
        # redundant but also helps me known which values to update for test data feature space
        X_test.iloc[:,(n_features*(n_time_steps+pot))-1].update(m_pred_test)
        
    if (n_time_steps == threshold):
        m.save_weights(final_file_path)
        
    X_train_all, y_train_all = provide_all_data(X_train, y_test, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_train_all, masked_y_train_all = prepare_for_mask(X_train_all, y_train_all)
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    # predict for all values 
    # use the masking list to known which of these should be used as imputation values
    all_train_pred = round_off_EDSS(m.predict(masked_X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))

    return X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test 

def update_previous_predictions(n_time_steps, all_train_pred, all_test_pred):
    
    m_pred_train = np.array(all_train_pred[:,n_time_steps-1,:]).flatten()
    m_pred_test = np.array(all_test_pred[:,n_time_steps-1,:]).flatten()
    
    return m_pred_train, m_pred_test

def impute_previous_predictions(n_time_steps, X_train, X_test,y_train,
                                m_pred_train, m_pred_test, mask_list, 
                                n_features = n_features, pot = pot):
    
    n_inputs = n_features*pot
    
    mask_train = mask_list[n_time_steps-1][0]
    mask_test = mask_list[n_time_steps-1][1]
    
    m_pred_train = pd.Series(m_pred_train[~mask_train], index = y_train[~mask_train].index)
    m_pred_test = pd.Series(m_pred_test[~mask_test], index = y_test[~mask_test].index)
    
    y_train.iloc[:, n_time_steps-1].update(m_pred_train)
    
    if n_time_steps < len(y_train.columns): # 36
        
        X_train.iloc[:,(n_features*(n_time_steps+pot))-1].update(m_pred_train)
        X_test.iloc[:,(n_features*(n_time_steps+pot))-1].update(m_pred_test)
    
    return X_train, X_test, y_train # y test does not get imputed

# Part 2

- Iterate predictive imputation for all time steps 
- Update all imputated values for each iteration

In [6]:
def exhaustive_imputation(X_train, y_train, X_test, y_test, n_features=n_features, pot = pot):
    
    """Running the model after we've impute for the whole feature space"""
    
    n_inputs = n_features*pot
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()

    m = rnn_model(n_time_steps, n_inputs)
    
    # load weights from previous model to establish continuity 
    m.load_weights(final_file_path)
    
    history = m.fit(X_train_all, y_train_all,
                    validation_split = 0.2, 
                    batch_size = X_train_all.shape[0], # of training instances == batch_size 
                    epochs=1,
                    shuffle=True,
                   callbacks = callbacks_list_final) # should save best weights 
    
    m.save_weights(final_file_path)
    
    all_train_pred = round_off_EDSS(m.predict(X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))
    
    val_loss = history.__dict__['history']['val_loss']
    
    return all_train_pred, all_test_pred, val_loss, m

def iterate_exhaustive_imputation(n_epochs, X_train, y_train, X_test, y_test):
    """
    We divide our RNN into different 1 epoch models 
    in order to iteratively update the imputations
    """
    val_loss_list = []
    
    for i in range(n_epochs):
        
        print('EPOCH: ', str(i))
        all_train_pred, all_test_pred, val_loss, model = exhaustive_imputation(X_train, y_train, 
                                                                               X_test, y_test)

        val_loss_list.append(val_loss)
    
        for i in range(1, all_train_pred.shape[1]): # -1 

                    m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, 
                                                                            all_test_pred)    
                    
                    X_train, X_test, y_train = impute_previous_predictions(i, X_train, X_test,
                                                                           y_train, m_pred_train, 
                                                                           m_pred_test, mask_list)
    
    return np.min(val_loss_list), model

# Part 3

- Generate final imputation and return imputed datasets

In [7]:
def best_weights_imputation(X_train, y_train, X_test, y_test, n_features=n_features, pot = pot):
    
    """
    Returns EDSS imputations using best model weights
    Running the model after we've impute for the whole feature space"""
    n_inputs = n_features*pot
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    # we do want to mask the instances of the testing data where there are missing targets
    # so that it does not affect our val_loss
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()

    m = rnn_model(n_time_steps, n_inputs)
    # load weights from best weights model to generate final imputation values 
    m.load_weights(best_file_path)
    
    all_train_pred = round_off_EDSS(m.predict(X_train_all))
    all_test_pred = round_off_EDSS(m.predict(masked_X_test_all))
    
    return all_train_pred, all_test_pred

def return_imputed_data(X_train, X_test, y_train, y_test):
    
    """
    Part 2
    Returns final data sets imputed with best model weights
    [Feature EDSS (train and test); target EDSS (train)]
    """
    
    val_loss_list = []
    
    all_train_pred, all_test_pred = best_weights_imputation(X_train, y_train, X_test, y_test)
    
    for i in range(1, all_train_pred.shape[1]): # -1 

                m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, all_test_pred)    
                    
                X_train, X_test, y_train = impute_previous_predictions(i, X_train, X_test,y_train, 
                                                                       m_pred_train, m_pred_test, 
                                                                       mask_list)
    
    n_time_steps = len(y_test.columns)
    
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    return X_train, X_test, y_train, y_test

# Import Data

In [8]:
X = pd.read_csv('../../../data/pre_imputation_data/X_2_years|6_months.csv', index_col = 0)
y = pd.read_csv('../../../data/pre_imputation_data/y_2_years|6_months.csv', index_col = 0)

#n_time_steps = len(y.columns)
n_features = X.columns.tolist().index("EDSS_0")+1
pot = 4
n_inputs = n_features * pot
n_units = 128
skf = KFold(n_splits=5, shuffle=True, random_state=42)

print("The input length of the training data will be", pot, "time slices, separated by 6 month intervals")
print(n_features, "featues comprise one time slice")

# Fill in first X
first_X_ffill = X[['EDSS_0','EDSS_6','EDSS_12','EDSS_18']].fillna(method = 'ffill',  axis = 1)
for col in first_X_ffill.columns:
    if col in X.columns:
         X[col] = first_X_ffill[col]
        
# Drop all missing values from EDSS_0 which is our first X feature predictor
mask = X.index.isin(X[['EDSS_0','EDSS_6', 'EDSS_12','EDSS_18']].dropna().index.tolist())

X_og, y_og = X[mask], y[mask]

The input length of the training data will be 4 time slices, separated by 6 month intervals
115 featues comprise one time slice


# Divide Data into Training and Testing

In [9]:
for f_ix in range(1,5):

    train_ix = []
    test_ix = []
    for train, test in skf.split(X_og.index):
        train_ix.append(train)
        test_ix.append(test)

    X_train_og, y_train_og = X_og.iloc[train_ix[f_ix], :].copy(), y_og.iloc[train_ix[f_ix], :].copy()
    X_test_og, y_test_og = X_og.iloc[test_ix[f_ix], :].copy(), y_og.iloc[test_ix[f_ix], :].copy()

    # Run algorithm

    mask_list = []

    X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(1, X_train_og, 
                                                                                        y_train_og, 
                                                                                        X_test_og, 
                                                                                        y_test_og)

    mask_list.append((mask_train, mask_test)) # these tell me what values are missing in each column 

    # Sanity Check

    print("There are missing values for EDSS in the X feature space:", pd.isna(X_train['EDSS_24']).any())
    print("There are missing values for EDSS in the y target space:", pd.isna(y_train['EDSS_24']).any())

    for r in range(2, len(y_train.columns)+1):

        X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(r, X_train, y_train, X_test, y_test)

        mask_list.append((mask_train, mask_test))

        if (all_train_pred.shape[1] > 1):

            # update all previous predictions
            for col in range(1,all_train_pred.shape[1]):
                # for each column of the training and testing EDSS
                m_pred_train, m_pred_test = update_previous_predictions(col, all_train_pred, all_test_pred)    
                X_train, X_test, y_train = impute_previous_predictions(col, X_train, X_test, y_train,m_pred_train, m_pred_test, mask_list)


    # X_train should go up to 210

    save_temp = (X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test)
    save_list_temp = mask_list.copy()

    X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = save_temp 
    mask_list = save_list_temp

    # Run final model for training with whole feature space and training targets

    best, model = iterate_exhaustive_imputation(100, X_train, y_train, X_test, y_test) # use original y test

    best

    # Generate optimal imputation 

    X_train, X_test, y_train, y_test =  return_imputed_data(X_train, X_test, y_train, y_test) # use original y test

    # Use original unimputed test data and save data

    X_test_no_imputation, y_test_no_imputation = X_og.iloc[test_ix[f_ix], :].copy(), y_og.iloc[test_ix[f_ix], :].copy()

    # save data folds
    a,b,c,d = def_train_name(f_ix)

    X_train.to_csv(a)
    X_test_no_imputation.to_csv(b)
    y_train.to_csv(c)
    y_test_no_imputation.to_csv(d)
    
    X_test.to_csv("unused_data_folds/X_test_"+str(f_ix+1)+".csv")
    y_test.to_csv("unused_data_folds/y_test_"+str(f_ix+1)+".csv")

TRAIN
NaN
X reshaped is (48, 1, 460)
y reshaped is (48, 1, 1)
Full
X reshaped is (345, 1, 460)
y reshaped is (345, 1, 1)
TEST
NaN
X reshaped is (11, 1, 460)
y reshaped is (11, 1, 1)
Full
X reshaped is (87, 1, 460)
y reshaped is (87, 1, 1)
Train on 276 samples, validate on 69 samples
Epoch 1/1
276/276 [==============================] - 2s 6ms/step - loss: 1.8608 - val_loss: 1.3103
X reshaped is (393, 1, 460)
y reshaped is (98, 1, 1)
X reshaped is (98, 1, 460)
y reshaped is (98, 1, 1)
There are missing values for EDSS in the X feature space: False
There are missing values for EDSS in the y target space: False
TRAIN
NaN
X reshaped is (79, 2, 460)
y reshaped is (79, 2, 1)
Full
X reshaped is (314, 2, 460)
y reshaped is (314, 2, 1)
TEST
NaN
X reshaped is (26, 2, 460)
y reshaped is (26, 2, 1)
Full
X reshaped is (72, 2, 460)
y reshaped is (72, 2, 1)
Train on 251 samples, validate on 63 samples
Epoch 1/1
251/251 [==============================] - 2s 7ms/step - loss: 1.5611 - val_loss: 1.0418
X 

Train on 145 samples, validate on 37 samples
Epoch 1/1
145/145 [==============================] - 2s 14ms/step - loss: 0.9095 - val_loss: 1.1018
X reshaped is (393, 17, 460)
y reshaped is (98, 17, 1)
X reshaped is (98, 17, 460)
y reshaped is (98, 17, 1)
TRAIN
NaN
X reshaped is (227, 18, 460)
y reshaped is (227, 18, 1)
Full
X reshaped is (166, 18, 460)
y reshaped is (166, 18, 1)
TEST
NaN
X reshaped is (81, 18, 460)
y reshaped is (81, 18, 1)
Full
X reshaped is (17, 18, 460)
y reshaped is (17, 18, 1)
Train on 132 samples, validate on 34 samples
Epoch 1/1
132/132 [==============================] - 2s 15ms/step - loss: 0.8929 - val_loss: 1.0206
X reshaped is (393, 18, 460)
y reshaped is (98, 18, 1)
X reshaped is (98, 18, 460)
y reshaped is (98, 18, 1)
TRAIN
NaN
X reshaped is (225, 19, 460)
y reshaped is (225, 19, 1)
Full
X reshaped is (168, 19, 460)
y reshaped is (168, 19, 1)
TEST
NaN
X reshaped is (82, 19, 460)
y reshaped is (82, 19, 1)
Full
X reshaped is (16, 19, 460)
y reshaped is (16, 1

EPOCH:  0
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.6327 - val_loss: 0.8144

Epoch 00001: val_loss improved from inf to 0.81442, saving model to best/best_weights_2y.hdf5
EPOCH:  1
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.6351 - val_loss: 0.8155

Epoch 00001: val_loss did not improve from 0.81442
EPOCH:  2
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.6036 - val_loss: 1.0757

Epoch 00001: val_loss did not improve from 0.81442
EPOCH:  3

314/314 [==============================] - 2s 7ms/step - loss: 0.6741 - val_loss: 0.9748

Epoch 00001: val_loss did not improve from 0.73568
EPOCH:  26
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.5947 - val_loss: 0.7566

Epoch 00001: val_loss did not improve from 0.73568
EPOCH:  27
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.5996 - val_loss: 0.7880

Epoch 00001: val_loss did not improve from 0.73568
EPOCH:  28
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms

Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.6289 - val_loss: 0.7944

Epoch 00001: val_loss did not improve from 0.72323
EPOCH:  52
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.5627 - val_loss: 0.8070

Epoch 00001: val_loss did not improve from 0.72323
EPOCH:  53
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 7ms/step - loss: 0.6349 - val_loss: 0.7675

Epoch 00001: val_loss did not improve from 0.72323
EPOCH:  54
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoc

...

X reshaped is (94, 23, 460)
y reshaped is (94, 23, 1)
TEST
NaN
X reshaped is (92, 23, 460)
y reshaped is (92, 23, 1)
Full
X reshaped is (6, 23, 460)
y reshaped is (6, 23, 1)
Train on 75 samples, validate on 19 samples
Epoch 1/1
75/75 [==============================] - 2s 21ms/step - loss: 0.8654 - val_loss: 0.7789
X reshaped is (393, 23, 460)
y reshaped is (98, 23, 1)
X reshaped is (98, 23, 460)
y reshaped is (98, 23, 1)
TRAIN
NaN
X reshaped is (299, 24, 460)
y reshaped is (299, 24, 1)
Full
X reshaped is (94, 24, 460)
y reshaped is (94, 24, 1)
TEST
NaN
X reshaped is (93, 24, 460)
y reshaped is (93, 24, 1)
Full
X reshaped is (5, 24, 460)
y reshaped is (5, 24, 1)
Train on 75 samples, validate on 19 samples
Epoch 1/1
75/75 [==============================] - 2s 21ms/step - loss: 0.7551 - val_loss: 1.0127
X reshaped is (393, 24, 460)
y reshaped is (98, 24, 1)
X reshaped is (98, 24, 460)
y reshaped is (98, 24, 1)
TRAIN
NaN
X reshaped is (324, 25, 460)
y reshaped is (324, 25, 1)
Full
X reshap

EPOCH:  9
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 6s 19ms/step - loss: 0.6065 - val_loss: 0.9008

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  10
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 16ms/step - loss: 0.6064 - val_loss: 0.7096

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  11
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 16ms/step - loss: 0.5557 - val_loss: 0.7683

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  12
X reshaped is (393, 33, 460)
y resha


Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  35
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 15ms/step - loss: 0.5711 - val_loss: 0.7980

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  36
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 15ms/step - loss: 0.5435 - val_loss: 0.8222

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  37
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 15ms/step - loss: 0.5741 - val_loss: 0.7965

Epoch 00001: val_loss did not improve from 0.


Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  61
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 8s 25ms/step - loss: 0.4710 - val_loss: 0.8578

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  62
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 6s 18ms/step - loss: 0.6441 - val_loss: 0.6066

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  63
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 4s 13ms/step - loss: 0.4654 - val_loss: 0.6418

Epoch 00001: val_loss did not improve from 0.


Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  87
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 5s 15ms/step - loss: 0.5458 - val_loss: 0.7301

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  88
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 8s 24ms/step - loss: 0.5045 - val_loss: 0.6124

Epoch 00001: val_loss did not improve from 0.42944
EPOCH:  89
X reshaped is (393, 33, 460)
y reshaped is (393, 33, 1)
X reshaped is (98, 33, 460)
y reshaped is (98, 33, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 6s 18ms/step - loss: 0.4875 - val_loss: 0.6314

Epoch 00001: val_loss did not improve from 0.